In [60]:
import numpy as np
import pandas as pd
import random as rd
import copy
import time

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Projet Element Logiciel

## Autour de l'article, présentation du projet

<p style="text-align:justify">
L'idée du projet est de paralléliser l'algorithme word2vec. On s'appuie pour cela sur l'article de Ji, Shihao, et al. <i>Parallelizing word2vec in shared and distributed memory</i>. Nous avons utilisé <i> pycuda </i> pour effectuer la programmation GPU. Dans ce notebook on utilise donc le langage Python comme langage d'environnement : on s'en sert principalement pour préparer les données, présenter les résultats et exécuter les fonctions à paralléliser. L'algorithme word2vec est en revanche codé en C.
</p>

<p style="text-align:justify">
L'algorithme word2vec permet de représenter les mots d'un texte dans un espace de petite dimension (typiquement R<sup>d</sup> avec d de l'ordre de 10). Il permet donc de passer d'une représentation des données sous forme one-hot (où la taille de l'espace des données est égale à la taille du vocabulaire du corpus de texte étudié) à une représentaion dans R<sup>d</sup>. L'idée est que, des mots proches sémantiquement (on les suppose proches si ils sont proches dans le texte) doivent être proches dans l'espace de projection R<sup>d</sup> (au sens euclidien cette fois).
</p>

<p style="text-align:justify">
L'architecture de word2vec est un réseau de neurones à une couche cachée. Si l'on souhaite que notre espace de projection soit R<sup>d</sup>, on choisira d neurones dans la couche cachée. Les couches d'entrée et de sortie sont de taille V où V est la taille du vocabulaire du corpus de texte étudié. Pour construire notre matrice de projection dans R<sup>d</sup>, on entraîne ce réseau de neurones via une tâche de prédiction annexe. Pour un mot en entrée, le réseau doit prédire le où les mots les plus proches sémantiquement. La matrice des poids de passage de la couche d'input à la couche cachée sera alors notre matrice de projection. On construit donc un ensemble d'entraînement (X,Y) où on fixe la taille N du contexte utilisé et les couples (Xi,Yi) sont determinés de la manière suivante. Pour un mot Xi du texte et pour les mots Yi1, Yi2,...,YiN qui sont autour du mot Xi dans le texte les couples (Xi,Yi1),...(Xi,YiN) forment la base d'entraînement.
</p>

<p style="text-align:justify">
L'entraînement de ce réseau nécessite l'implémentation de l'algorithme de descente de gradient. L'article met en avant les inconvénients de cet algorithme dans ce cas précis, en particulier l'évaluation de la fonction de perte et de ses gradients est très gourmande (le nombre de calculs étant proportionnel au nombre de mots dans le texte). On préférera donc utiliser la méthode de descente de gradient stochastique (avec negative sampling) qui n'utilise qu'une faible proportion des mots du texte à chaque évaluation des oracles. A chaque étape la méthode de gradient stochastique met à jour les poids correspondant à un mot Xi et à un mot de son contexte Yij. Le negative sampling consiste à prendre des mots au hasard dans le texte à chaque étape et à mettre à jour les poids correspondant en considérant que ces mots ne sont pas dans le contexte de Xi.
</p>

<p style="text-align:justify">
Dans l'article, les auteurs proposent, pour accélerer l'implémentation de la descente de gradient stochastique, de paralléliser les boucles de mise à jour des poids du réseau (voir section III de l'article). Cela risque de créer des conflits (puisque les mêmes poids risquent d'être mis à jour au même moment) mais on s'en affranchit, cela peut réduire théoriquement les taux de convergence de l'algorithme mais permet de gagner en vitesse (d'un facteur du nombre de parallélisation). On observe que pour des grands textes et à condition de ne pas utiliser un nombre de threads trop élevé, les conflits sont rares et n'empechent pas la convergence. L'algorihme que l'on parallélise est l'implémentation d'Hogwild (voir <i>Algorithm 1</i> de l'article).
</p>

<p style="text-align:justify">
La deuxième amélioration proposée dans l'article est double. Elle consiste à exploiter plus encore les propriétés locales des informations utilisées à chaque boucle de mise à jour. L'idée est de partager le <i>negative sampling</i> entre les mots d'un même contexte. Cela va permettre de passer des opérations de produits scalaires (opérations BLAS-1)  utilisés massivement dans l'implémentation d'Hogwild à des produits matriciels (opérations BLAS-3) en théorie plus efficaces en utilisant des bibliothèques performatantes d'algèbre linéaire.
</p>

<p>
On s'attache dans ce projet à mettre en place ces deux techniques sur un petit jeu de données et d'observer les gains effectivement obtenus.
</p>

## Préparation des données

<p style="text-align:justify">
On utilise comme jeu de données, un extrait des Misérables de Victor Hugo traduit en anglais. Le texte a été nétoyé de tous ces caractères spéciaux. Cet extrait contient 39.079 mots dont 6128 mots différents. On commence par encoder le texte sous forme one-hot : matrice de taille n*V où n est le nombre de mots du texte et V le nombre de mots différents et l'élément (i,j) de cette matrice vaut 1 si le ième mot est identifié comme le jème mot du vocabulaire, 0 sinon. Cette matrice est notre base de travail pour l'entraînement du réseau de neurones. Nous avons trouvé plus pratique de travailler avec une forme équivalente : un vecteur de taille n où le ième terme est l'identifiant (un entier entre 1 et V) du ième mot du texte.
</p>

In [2]:
#Chargement du jeu de données
text  = open("data/LesMiserables_cleaned.txt", "r")
text = text.read()
text = text.split(" ")
text_serie = pd.Series(text)
text_serie.head(10)

0          e
1    address
2     marius
3     turned
4       pale
5        all
6        the
7      blood
8     flowed
9       back
dtype: object

In [3]:
#One-hot encoding
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
text_lb = LabelEncoder()
X_hot = text_lb.fit_transform(text_serie.values)

In [4]:
np.shape(X_hot)
#Le ième mot du texte est le mot j => X_hot_ij = 1
#dimension de X_hot : n*V
#n : nombre de mot dans le texte
#V : nombre de mots différents dans le texte

(100648,)

In [5]:
X_hot

array([2955,  180, 5635, ...,  303,  608,    0])

In [6]:
np.max(X_hot)

10215

In [7]:
print(text_lb.inverse_transform([10]))
print(text_lb.transform(["let"]))
#Il est plus simple de représenter le texte sous cette forme
#X_i est l'identifiant du ième mot du texte

['1776']
[5317]


## Hogwild algorithm in Python

<p style="text-align:justify">
On commence par coder l'algorithme sous Python. Cette première implémentation de la méthode ne sera donc parallélisée et nous permettra de comparer les résultats et la vitesse de calcul avec le code parallélisé sous pycuda. On rédige volontairemnt un code non utilisé on utilise pas par exemple les bibliothèques de calcul d'algèbre linéaire pour le calcul matriciel. Cela nous permettra de comparer les méthodes sous la même base de travail et d'observer correctement les gains engendrés par les améliorations de l'article.
</p>

In [ ]:
# On commence par rédiger le code sans parallélisation 
def sig(x):
    return 1/(1+np.exp(-x))

def hog_loop(Min,Mout,alpha,wout,Nwin,negative):
    #Min matrice dans V*d
    #Mout matrice dans V*d
    #wout un élément de 0:V-1
    #Nwin n entiers de 0:V-1
    V,d = Min.shape
    N = Nwin.shape[0]
    for i in range(N):
        input_word = Nwin[i]
        temp = np.array([0.0]*d)
        for k in range(negative+1):
            if k == 0:
                target_word = wout
                label = 1
            else : 
                target_word = rd.randint(0,V-1)
                label = 0
            inn = 0
            for j in range(d):
                inn = inn + Min[input_word,j]*Mout[target_word,j]
            err = label - sig(inn)
            for j in range(d):
                temp[j] = temp[j] + err*Mout[target_word,j]
            for j in range(d):
                Mout[target_word,j] = Mout[target_word,j] + alpha*err*Min[input_word,j]
        for j in range(d):
            Min[input_word,j] = Min[input_word,j] + alpha*temp[j]
    return(Min,Mout)
    

In [ ]:
def train_word2vec(X,params):
    
    N = params['N']
    n_epochs = params['n_epochs']
    alpha = params['alpha']
    negative = params['negative']
    d = params['d']
    
    V = np.max(X)+1
    n_words = np.shape(X)[0]
    
    Min = np.array([rd.random() for i in range(V*d)]).reshape((V,d))
    Mout = np.array([rd.random() for i in range(V*d)]).reshape((V,d))
    
    for i in range(n_epochs):
        
        epochs_order = np.array([t for t in range(N,n_words-N)])
        np.random.shuffle(epochs_order)
        
        for j in epochs_order:
            
            wout = X[j]
            
            Nwin = []
            for k in range(N):
                Nwin.append(X[j-N+k])
            for k in range(N):
                Nwin.append(X[j+k+1])
            Nwin = np.array(Nwin)
                
            Min, Mout = hog_loop(Min,Mout,alpha,wout,Nwin,negative)
            
    
    return Min,Mout

In [17]:
params = {
    "N" : 2,
    "n_epochs" : 10,
    "alpha" : 10**(-3),
    "negative" : 4,
    "d" : 10
}


In [ ]:
t=time.time()
Min,Mout = train_word2vec(X=X_hot, params=params)
t=time.time()-t
print("temps pour effectuer %d epochs : %s" % (params['n_epochs'],t))
print(Min)

In [8]:
def prevision(mot,Min,Mout,text_lb,p):
    
    index = text_lb.transform([mot])[0]
    sub_Min = Min[index]
    
    prev = np.dot(sub_Min,Mout.T)
    prev = np.exp(prev)/(np.sum(np.exp(prev)))
    
    index_prev = np.flip(np.argsort(prev))[:p]

    word_prev = text_lb.inverse_transform(index_prev)
    
    return word_prev
   

In [ ]:
print(prevision("let",Min,Mout,text_lb,4))
print(prevision("it",Min,Mout,text_lb,4))
print(prevision("be",Min,Mout,text_lb,4))
print(prevision("wisdom",Min,Mout,text_lb,4))

### in Pycuda

RMQ Dupre : 

regarder 
// ++i i++
   for(auto it: temp)
       *it = 0;
    
for(float* it = temp; it != ; ++it)
      *it = 0;
    

In [18]:
mod = SourceModule("""

#include <time.h>
#include <stdlib.h>
#include <stdio.h>
#include <curand.h>
#include <math.h>

__device__ void loop(float *Min, float *Mout, float alpha, int *Nwin, int wout, int V, int d, int N, int negative, int random){
    
    /*
    HOGWILD LOOP
    
    float *Min : initialisé en dehors de pycuda, poids de la PREMIERE couche du RN (attention float * et non float **) (V*d)
    float *Mout : initialisé en dehors de pycuda, poids de la DEUXIEME couche du RN (attention float * et non float **) (V*d)
    float alpha : learning rate
    int *Nwin : 2*N input du contexte de l'output
    int wout : output
    int V : taille du vocabulaire (nombre de mots différents)
    int d : taille de l'espace de représentation
    int N : taille du contexte utilisé pour l'apprentissage (de chaque coté donc 2*N en tout)
    int negative : nb de negative utilisé pour l'apprentissage
    int random : un entier aléatoire (difficulté pour générer de l'aléat sur le device) pour les negative
    
    Une boucle de l'Algo 1 de l'article. Permet de mettre à jour Min et Mout pour un wout (et son contexte associé).
    */

    /* Init variables */
    float* temp;
    temp = (float *)malloc(sizeof(float)*d);
    int label;
    int target_word;
    float inn;
    float err;
    int r;
    
    /* Boucle principale sur les 2*N inputs*/
    for(int i=0;i<2*N;i++){
        int input_word = Nwin[i];
        for(int j=0;j<d;j++){
            temp[j]=0;
        }
        for(int k=0; k<negative+1;k++){
            if (k==0){
                target_word = wout;
                label = 1;
            } else {
                /* negative sampling */
                r = (1664525*r+1013904223) % 4294967296;
                target_word = r%V;
                label = 0;
            }
            inn = 0;
            for(int j=0;j<d;j++){
                inn = inn + Min[input_word*d+j]*Mout[target_word*d+j];
            }
            err = label-(1/(1+exp(-inn)));
            for(int j=0;j<d;j++){
                temp[j] = temp[j]+err*Mout[target_word*d+j];
            }
            for(int j=0;j<d;j++){
                Mout[target_word*d+j] = Mout[target_word*d+j]+alpha*err*Min[input_word*d+j];
            }  
        }
        for(int j=0;j<d;j++){
            Min[input_word*d+j] = Min[input_word*d+j]+alpha*temp[j]; 
        }
    }
    
    free(temp);
}


__global__ void parallel(float *Min, float* Mout, int random, int* cst_int, float* cst_float, int* targets, int* contexts) {

    /*
    Parallélisation
    float *Min : Min initialisé
    float *Mout : Mout initialisé
    int* random : un entier aléatoire généré hors du device
    int* cst_int : les constantes entières utiles (V,d,negative,N)
    float* cst_float : les constantes float utiles (alpha)
    int* targets : on va travailler sur wout=targets[idx]
    int* context : le contexte associé sera contexts[idx]
    */

    /*préparation des paramètres pour la fonction loop*/
    
    
    /*constantes entières passés depuis le code python*/
    int V = cst_int[0];
    int d = cst_int[1];
    int negative = cst_int[2];
    int N = cst_int[3];

    /*constante float passé depuis python*/
    float alpha = cst_float[0];
    
    /*l'index correspond au thread et indique simplement le wout sur lequel on travaille*/
    int index = threadIdx.x; 
    int wout = targets[index];
    
    /*Context de wout*/
    int *Nwin;
    Nwin = (int *)malloc(sizeof(int)*N*2);
    for (int i=0;i<2*N;i++){
        Nwin[i] = contexts[index*2*N+i];
    }
    
    random = random+index;
    
    loop(Min,Mout,alpha,Nwin,wout,V,d,N,negative,random);
    
    free(Nwin);
}
""")

In [50]:
def train_word2vec_parallel(X, params, func):
    N = params['N']
    n_epochs = params['n_epochs']
    alpha = params['alpha']
    negative = params['negative']
    d = params['d']
    
    V = np.max(X)+1
    n_words = np.shape(X)[0]
    
    cst_int = np.array([V,d,negative,N])
    cst_float = np.array([alpha])
    
    cst_int = cst_int.astype(np.int32)
    cst_float = cst_float.astype(np.float32)
    
    Min = np.array([rd.random() for i in range(V*d)]).reshape((V,d))
    Mout = np.array([rd.random() for i in range(V*d)]).reshape((V,d))
    
    Min = Min.astype(np.float32)
    Mout = Mout.astype(np.float32)
    
    for i in range(n_epochs):
        
        epochs_order = np.array([t for t in range(N,n_words-N)])
        np.random.shuffle(epochs_order)
        
        for j in range(n_words//1000+1):
            sub_epochs_order = epochs_order[j*1000:(j+1)*1000]
            
            n_threads = np.shape(sub_epochs_order)[0]
            
            contexts=[]
            targets=[]
            
            for k in sub_epochs_order : 
                targets.append(X[k])
                Nwin=[]
                for l in range(N):
                    Nwin.append(X[k-N+l])
                for l in range(N):
                    Nwin.append(X[k+l+1])
                contexts.append(Nwin)
            
            targets = np.array(targets)
            contexts = np.array(contexts)
            
            targets = targets.astype(np.int32)
            contexts = contexts.astype(np.int32)
            
            r = np.array([rd.randint(1,1000000)])
            r = r.astype(np.int32)
            
            Min_gpu = cuda.mem_alloc(Min.nbytes)
            Mout_gpu = cuda.mem_alloc(Mout.nbytes)
            r_gpu = cuda.mem_alloc(r.nbytes)
            cst_int_gpu = cuda.mem_alloc(cst_int.nbytes)
            cst_float_gpu = cuda.mem_alloc(cst_float.nbytes)
            targets_gpu = cuda.mem_alloc(targets.nbytes)
            contexts_gpu = cuda.mem_alloc(contexts.nbytes)

            cuda.memcpy_htod(Min_gpu, Min)
            cuda.memcpy_htod(Mout_gpu, Mout)
            cuda.memcpy_htod(r_gpu, r)
            cuda.memcpy_htod(cst_int_gpu,cst_int)
            cuda.memcpy_htod(cst_float_gpu,cst_float)
            cuda.memcpy_htod(contexts_gpu,contexts)
            cuda.memcpy_htod(targets_gpu,targets)
            
            func(Min_gpu, Mout_gpu, r_gpu, cst_int_gpu, cst_float_gpu, targets_gpu, contexts_gpu, block=(n_threads,1,1))
            
            cuda.memcpy_dtoh(Min, Min_gpu)
            cuda.memcpy_dtoh(Mout, Mout_gpu)


    
    return Min,Mout
    

### Tests

In [51]:
func = mod.get_function("parallel")

In [52]:
params = {
    "N" : 2,
    "n_epochs" : 10,
    "alpha" : 10**(-3),
    "negative" : 4,
    "d" : 10
}

In [53]:
t0=time.time()
Min, Mout = train_word2vec_parallel(X=X_hot, params=params, func=func)
dt=time.time()-t0
print("temps d'execution de %d epochs" % params["n_epochs"])
print(dt)
print(Min)

temps d'execution de 10 epochs
11.935495853424072
[[0.99326545 0.4685385  0.47332487 ... 0.2653461  0.8222725  0.6239358 ]
 [0.93242335 0.84805053 0.22458169 ... 0.7781303  0.39575273 0.5576866 ]
 [0.23221748 0.32908982 0.8396483  ... 0.93647116 0.6011322  0.37795612]
 ...
 [0.01812954 0.7847018  0.8054997  ... 0.5697654  0.0586516  0.09843464]
 [0.9143857  0.9028726  0.37772062 ... 0.54738826 0.61895263 0.76268846]
 [0.93561375 0.07274387 0.23634802 ... 0.4439356  0.45557424 0.6066696 ]]


In [54]:
print(prevision("jean",Min,Mout,text_lb,3))
print(prevision("valjean",Min,Mout,text_lb,3))
print(prevision("man",Min,Mout,text_lb,3))


['the' 'a' 'of']
['the' 'a' 'of']
['the' 'a' 'to']


## Paper amelioration

In [55]:
mod_2 = SourceModule("""


#include <stdlib.h>
#include <stdio.h>
#include <curand.h>
#include <math.h>


__device__ void multiplication(float* A, float* B, float* C, int* dim){
    
    int lA = dim[0];
    int cA = dim[1];
    int lB = dim[2];
    int cB = dim[3];
    
    if (cA!=lB){
        return ;
    }
    
    for(int i=0; i<lA ; i++){
        for(int j=0; j<cB ; j++){
            C[i*cB+j] = 0;
        }
    }
    
    for(int i=0; i<lA ; i++){
        for(int j=0; j<cB ; j++){
            for(int k=0 ; k<cA ; k++){
                C[i*cB+j] = C[i*cB+j] + A[i*cA+k]*B[k*cB+j];
            }
        }
    }
}


__device__ void transpose(float* A, float* TA, int* dim){
    
    int lA = dim[0];
    int cA = dim[1];
      
    for(int i=0; i<cA ; i++){
        for(int j=0; j<lA ; j++){
            TA[i*lA+j] = A[j*cA+i];
        }
    }
}


__device__ void loop(float *Min, float *Mout, float alpha, int *Nwin, int wout, int V, int d, int N, int negative, int r){


    /*Calcul des target_words (wout et negative sample)*/
    int *target_words;
    target_words = (int *)malloc(sizeof(int)*(negative+1));
    for(int i=0; i<negative+1;i++){
        if (i==0){
            target_words[i] = wout;
        } else {
            r = (1664525*r+1013904223) % 4294967296;
            target_words[i] = r%V;
        }
    }
    
    /*label*/
    float *label;
    label = (float *)malloc(sizeof(float)*(2*N*(negative+1)));
    for (int i=0; i<2*N ; i++){
        for (int j=0 ; j<negative+1;j++){
            if (j==0){
                label[i*(negative+1)+j] = 1;
            } else {
                label[i*(negative+1)+j] = 0;
            }
        }
    }
    
    /*sub_Min*/
    float* sub_Min;
    sub_Min = (float *)malloc(sizeof(float)*2*N*d);
    for (int i=0; i<2*N ; i++){
        for (int j=0 ; j<d ; j++){
            sub_Min[i*d+j] = Min[Nwin[i]*d+j];
        }
    }
    
    /*sub_Mout*/
    float* sub_Mout;
    sub_Mout = (float *)malloc(sizeof(float)*(negative+1)*d);
    for (int i=0; i<negative+1 ; i++){
        for (int j=0 ; j<d ; j++){
            sub_Mout[i*d+j] = Mout[target_words[i]*d+j];
        }
    }

    /*sub_Mout_transpose*/
    float* sub_Mout_transpose;
    sub_Mout_transpose = (float *)malloc(sizeof(float)*d*(negative+1));
    int dim_sub_Mout[2];
    dim_sub_Mout[0] = negative+1;
    dim_sub_Mout[1] = d;
    transpose(sub_Mout, sub_Mout_transpose, dim_sub_Mout);
    
    /*INN*/
    float* INN;
    INN = (float *)malloc(sizeof(float)*2*N*(negative+1));
    int dim[4];
    dim[0] = 2*N;
    dim[1] = d;
    dim[2] = d;
    dim[3] = negative+1;
    multiplication(sub_Min, sub_Mout_transpose, INN, dim);
    
    /*ERR*/
    float* ERR;
    ERR = (float *)malloc(sizeof(float)*2*N*(negative+1));
    for (int i=0; i<2*N ; i++){
        for (int j=0 ; j<negative+1;j++){
            ERR[i*(negative+1)+j] = label[i*(negative+1)+j] - (1/(1+exp(-1*INN[i*(negative+1)+j])));
        }
    }  
    
    /*TEMP*/
    float* TEMP;
    TEMP = (float *)malloc(sizeof(float)*2*N*d);
    dim[0] = 2*N;
    dim[1] = negative+1;
    dim[2] = negative+1;
    dim[3] = d;
    multiplication(ERR, sub_Mout, TEMP, dim);
    
    /*MAJ Mout*/
    float* ERR_transpose_alpha;
    ERR_transpose_alpha = (float *)malloc(sizeof(float)*(negative+1)*2*N);
    for (int i=0; i<negative+1 ; i++){
        for (int j=0 ; j<2*N;j++){
            ERR_transpose_alpha[i*2*N+j] = alpha*ERR[j*(negative+1)+i]; 
        }
    }
    float* MAJ_Mout;
    MAJ_Mout = (float *)malloc(sizeof(float)*(negative+1)*d);
    dim[0] = negative+1;
    dim[1] = 2*N;
    dim[2] = 2*N;
    dim[3] = d;
    multiplication(ERR_transpose_alpha, sub_Min, MAJ_Mout, dim);
    for (int i=0; i<negative+1 ; i++){
        for (int j=0 ; j<d;j++){
            Mout[target_words[i]*d+j] = Mout[target_words[i]*d+j] + MAJ_Mout[i*d+j]; 
        }
    }
    
    /*MAJ Min*/
    for(int i = 0;i<2*N;i++){
        for(int j = 0;j<d;j++){
            Min[Nwin[i]*d+j] = Min[Nwin[i]*d+j] + alpha*TEMP[i*d+j];
        }
    }
    
    free(target_words);
    free(label);
    free(sub_Min);
    free(sub_Mout);
    free(sub_Mout_transpose);
    free(INN);
    free(ERR);
    free(TEMP);
    free(ERR_transpose_alpha);
    free(MAJ_Mout);

}



__global__ void parallel(float *Min, float* Mout, int random, int* cst_int, float* cst_float, int* targets, int* contexts) {

    /*constantes entières passés depuis le code python*/
    int V = cst_int[0];
    int d = cst_int[1];
    int negative = cst_int[2];
    int N = cst_int[3];

    /*constante float passé depuis python*/
    float alpha = cst_float[0];
    
    /*l'index correspond au thread et indique simplement le wout sur lequel on travaille*/
    int index = threadIdx.x; 
    int wout = targets[index];
    
    /*Context de wout*/
    int *Nwin;
    Nwin = (int *)malloc(sizeof(int)*N*2);
    for (int i=0;i<2*N;i++){
        Nwin[i] = contexts[index*2*N+i];
    }
    
    random = random+index;
    
    loop(Min,Mout,alpha,Nwin,wout,V,d,N,negative,random);
   
    free(Nwin);
    
}

""")

In [56]:
func_2 = mod_2.get_function("parallel")

In [57]:
t0=time.time()
Min, Mout = train_word2vec_parallel(X=X_hot, params=params, func=func_2)
dt=time.time()-t0
print("temps d'execution de %d epochs" % params["n_epochs"])
print(dt)
print(Min)

temps d'execution de 10 epochs
32.57986545562744
[[ 7.4137259e-01  4.7478074e-01  3.3267787e-01 ...  9.5184082e-01
   1.8321779e-01  9.3742126e-01]
 [ 7.9007107e-01  7.5255549e-01  4.6027899e-01 ... -1.2118397e-01
   1.7460804e-01  6.6338271e-01]
 [ 4.6469308e-02  4.8930153e-01  7.8742313e-01 ...  8.3109987e-01
   1.0589902e-01  8.2291704e-01]
 ...
 [-1.5304189e-02  6.4523506e-01  4.2101943e-01 ...  5.2118313e-01
   5.5230379e-01  1.0219754e-01]
 [ 7.5730002e-01  7.7641970e-01  9.0988910e-01 ...  1.4865802e-01
   7.2649352e-02  6.9265908e-01]
 [ 8.3669215e-02  6.8336993e-01  2.5119770e-01 ...  2.3244886e-01
   5.7150173e-04  1.8801883e-01]]


In [59]:
print(prevision("jean",Min,Mout,text_lb,3))
print(prevision("valjean",Min,Mout,text_lb,3))
print(prevision("woman",Min,Mout,text_lb,3))

['one' 'was' 'he']
['that' 'had' 'not']
['that' 'i' 'had']


# Tests

In [ ]:
X_hot

In [ ]:
text_lb.classes_

In [ ]:
text_lb.inverse_transform(np.array([0]*6126+[1]))